In [2]:
from tensorflow import keras 
import sys
import h5py
import numpy as np
clean_data_filename = './drive/MyDrive/Lab3/cl/valid.h5'
clean_data_filename_test = './drive/MyDrive/Lab3/cl/test.h5'
poisoned_data_filename = './drive/MyDrive/Lab3/bd/bd_valid.h5'
model_filename = 'bd_net.h5'

def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))
    return x_data, y_data

cl_x_valid, cl_y_valid = data_loader(clean_data_filename)
cl_x_test, cl_y_test = data_loader(clean_data_filename_test)
bd_x_valid, bd_y_valid = data_loader(poisoned_data_filename)

In [3]:
# import tensorflow
# tensorflow.config.experimental.list_physical_devices('GPU')

In [4]:
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tqdm import tqdm

def get_intermediate(model, layer_ind):
    intermediate_layer_model = keras.models.Model(inputs=model.input,
                                 outputs=model.layers[layer_ind].output)
    return intermediate_layer_model

class G(keras.Model):
    def __init__(self, B, B_prime):
        super(G, self).__init__()
        self.B = B
        self.B_prime = B_prime

    def call(self,data):
        y = np.argmax(self.B(data), axis=1)
        y_prime = np.argmax(self.B_prime(data), axis=1)
        tmpRes = np.array([y[i] if y[i] == y_prime[i] else 1283 for i in range(y.shape[0])])
        res = np.zeros((y.shape[0],1284))
        res[np.arange(tmpRes.size),tmpRes] = 1
        return res
      
def prune_defense(X):
    layer_ind = 6
    B = keras.models.load_model(model_filename)
    B_pruned = keras.models.load_model(model_filename)

    cl_label_p = np.argmax(B.predict(cl_x_valid), axis=1)
    original_accuracy = np.mean(np.equal(cl_label_p, cl_y_valid))*100
    print("orig accuracy clean: ",original_accuracy)

    bd_label_p = np.argmax(B.predict(bd_x_valid), axis=1)
    asr = np.mean(np.equal(bd_label_p, bd_y_valid))*100
    print("asr badnet: ",asr)

    intermediate_repr = get_intermediate(B, layer_ind)
    activations = intermediate_repr.predict(cl_x_valid)

    avg_activations = activations.mean(axis=(0,1,2))
    sorted_activations = np.argsort(avg_activations)
    #remove channels in decreasing order of avg_activations
    for channel in tqdm(sorted_activations):
        # Prune the last layer
        layer= B_pruned.layers[layer_ind-1]
        K.set_value(layer.kernel[:, :, :, channel], np.zeros_like(layer.kernel[:, :, :, channel]))
        K.set_value(layer.bias[channel], np.zeros_like(layer.bias[channel]))

        # B_pruned = delete_channels(B_pruned, B_pruned.layers[layer_ind-1], [i])
        cl_label_p = np.argmax(B_pruned.predict(cl_x_valid), axis=1)
        pruned_accuracy = np.mean(np.equal(cl_label_p, cl_y_valid))*100
        print('Validation accuracy after pruning channel', channel, ':', pruned_accuracy)
        if abs(original_accuracy - pruned_accuracy) > X:
            break

    goodnet = G(B, B_pruned)
    
    preds = goodnet(cl_x_test)
    cl_label_p = np.argmax(preds, axis=1)
    asr = np.mean(np.equal(cl_label_p, cl_y_test))*100
    print("clean test acc goodnet: ",asr)
    
    preds = goodnet(bd_x_valid)
    bd_label_p = np.argmax(preds, axis=1)
    asr = np.mean(np.equal(bd_label_p, bd_y_valid))*100
    print("asr goodnet: ",asr)
    return B_pruned

In [10]:
repaired_2 = prune_defense(2)

361/361 [==============================] - 1s 2ms/step
orig accuracy clean:  98.64899974019225
361/361 [==============================] - 1s 2ms/step
asr badnet:  100.0
361/361 [==============================] - 1s 2ms/step


  0%|          | 0/60 [00:00<?, ?it/s]

361/361 [==============================] - 1s 2ms/step


  2%|▏         | 1/60 [00:01<01:28,  1.50s/it]

Validation accuracy after pruning channel 0 : 98.64899974019225
361/361 [==============================] - 1s 2ms/step


...

 73%|███████▎  | 44/60 [01:10<00:27,  1.72s/it]

Validation accuracy after pruning channel 1 : 97.50584567420108
361/361 [==============================] - 1s 2ms/step


 73%|███████▎  | 44/60 [01:12<00:26,  1.65s/it]

Validation accuracy after pruning channel 29 : 95.75647354291158


clean test acc goodnet:  95.74434918160561
asr goodnet:  100.0


In [13]:
repaired_2.save("repaired_2", save_format='h5')

In [5]:
repaired_4 = prune_defense(4)

361/361 [==============================] - 8s 3ms/step
orig accuracy clean:  98.64899974019225
361/361 [==============================] - 1s 2ms/step
asr badnet:  100.0
361/361 [==============================] - 1s 2ms/step


  0%|          | 0/60 [00:00<?, ?it/s]

361/361 [==============================] - 1s 2ms/step


  2%|▏         | 1/60 [00:01<01:48,  1.84s/it]

Validation accuracy after pruning channel 0 : 98.64899974019225
361/361 [==============================] - 1s 2ms/step


...

 78%|███████▊  | 47/60 [01:15<00:19,  1.51s/it]

Validation accuracy after pruning channel 56 : 94.7172425738287
361/361 [==============================] - 1s 3ms/step


 78%|███████▊  | 47/60 [01:17<00:21,  1.65s/it]

Validation accuracy after pruning channel 46 : 92.09318437689443


clean test acc goodnet:  92.1278254091972
asr goodnet:  99.9913397419243


In [7]:
repaired_4.save("repaired_4.h5")

In [6]:
repaired_10 = prune_defense(10)

361/361 [==============================] - 1s 2ms/step
orig accuracy clean:  98.64899974019225
361/361 [==============================] - 1s 2ms/step
asr badnet:  100.0
361/361 [==============================] - 1s 2ms/step


  0%|          | 0/60 [00:00<?, ?it/s]

361/361 [==============================] - 1s 2ms/step


  2%|▏         | 1/60 [00:01<01:27,  1.48s/it]

Validation accuracy after pruning channel 0 : 98.64899974019225
361/361 [==============================] - 1s 2ms/step


...

 85%|████████▌ | 51/60 [01:25<00:15,  1.75s/it]

Validation accuracy after pruning channel 11 : 89.17467740538669
361/361 [==============================] - 1s 2ms/step


 85%|████████▌ | 51/60 [01:26<00:15,  1.70s/it]

Validation accuracy after pruning channel 54 : 84.43751623798389


clean test acc goodnet:  84.3335931410756
asr goodnet:  77.015675067117


In [8]:
repaired_10.save("repaired_10.h5")

In [ ]:
## Creating stats for report

In [17]:
def prune_defense_all(X):
    layer_ind = 6
    B = keras.models.load_model(model_filename)
    B_pruned = keras.models.load_model(model_filename)

    cl_label_p = np.argmax(B.predict(cl_x_valid), axis=1)
    original_accuracy = np.mean(np.equal(cl_label_p, cl_y_valid))*100
    print("orig accuracy clean: ",original_accuracy)

    bd_label_p = np.argmax(B.predict(bd_x_valid), axis=1)
    asr = np.mean(np.equal(bd_label_p, bd_y_valid))*100
    print("asr badnet: ",asr)

    intermediate_repr = get_intermediate(B, layer_ind)
    activations = intermediate_repr.predict(cl_x_valid)

    avg_activations = activations.mean(axis=(0,1,2))
    sorted_activations = np.argsort(avg_activations)
    l = len(sorted_activations)
    #remove channels in decreasing order of avg_activations
    i = 0
    for channel in (sorted_activations):
        # Prune the last layer
        layer= B_pruned.layers[layer_ind-1]
        K.set_value(layer.kernel[:, :, :, channel], np.zeros_like(layer.kernel[:, :, :, channel]))
        K.set_value(layer.bias[channel], np.zeros_like(layer.bias[channel]))

        # B_pruned = delete_channels(B_pruned, B_pruned.layers[layer_ind-1], [i])
        # cl_label_p = np.argmax(B_pruned.predict(cl_x_valid), axis=1)
        # pruned_accuracy = np.mean(np.equal(cl_label_p, cl_y_valid))*100
        # print('Validation accuracy after pruning channel', channel, ':', pruned_accuracy)
        
        goodnet = G(B, B_pruned)
        print(i/l)
        
        preds = goodnet(cl_x_test)
        cl_label_p = np.argmax(preds, axis=1)
        asr = np.mean(np.equal(cl_label_p, cl_y_test))*100
        print("clean test acc goodnet: ",asr)
        
        preds = goodnet(bd_x_valid)
        bd_label_p = np.argmax(preds, axis=1)
        asr = np.mean(np.equal(bd_label_p, bd_y_valid))*100
        print("asr goodnet: ",asr)
        i+=1

In [18]:
prune_defense_all(0)

361/361 [==============================] - 1s 2ms/step
orig accuracy clean:  98.64899974019225
361/361 [==============================] - 1s 2ms/step
asr badnet:  100.0
361/361 [==============================] - 1s 2ms/step
0.0
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.016666666666666666
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.03333333333333333
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.05
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.06666666666666667
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.08333333333333333
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.1
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.11666666666666667
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.13333333333333333
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.15
clean test acc goodnet:  98.62042088854248
asr goodnet:  100.0
0.16666